In [10]:
from dotenv import load_dotenv
import os
import geopandas as gpd

load_dotenv()

True

In [2]:
from minio import Minio

minio_client = Minio(
	endpoint=os.getenv('S3_ENDPOINT'),
	access_key=os.getenv('ACCESS_KEY_ID'),
	secret_key=os.getenv('SECRET_ACCESS_KEY'),
	secure=True,
)

In [33]:
# list all objects in the bucket

collections = list(minio_client.list_objects(os.getenv('BUCKET')))

count = 1
for collection in collections:
	print(collection.object_name, f"({count}/{len(collections)})")
	version = 1
	while True:
		# look for catalog.v{version}.parquet in bucket/collection
		object_name = f"{collection.object_name}catalog.v{version}.parquet"
		try:
			minio_client.stat_object(os.getenv('BUCKET'), object_name)
			print(object_name)
			# download the catalog
			minio_client.fget_object(os.getenv('BUCKET'), object_name, f"kk/{object_name}")
			# backup the catalog
			# minio_client.fput_object(os.getenv('BUCKET'), f"{object_name}.backup", f"kk/{object_name}")
			# read the catalog
			df = gpd.read_parquet(f"kk/{object_name}")
			# update the metadata
			for index, row in df.iterrows():
				for k, v in row.assets.items():
					v['href'] = v['href'].replace('//dev.api.eotdl.com', '//api.eotdl.com')
					# print(k, v)
				break
			# save the catalog
			df.to_parquet(f"kk/{object_name}")
			# upload the catalog
			minio_client.fput_object(os.getenv('BUCKET'), object_name, f"kk/{object_name}")
			version += 1
		except:
			# print(f"No {object_name} found")
			break
	count += 1
	# if count > 2:
	# 	break
		



test.txt (1/94)
645a26564d1c8b7b364ee631/ (2/94)
645a26564d1c8b7b364ee631/catalog.v1.parquet
645d1e4ec3060c653291ce87/ (3/94)
645d1e4ec3060c653291ce87/catalog.v1.parquet
645dff70c2f02fdd88c47b67/ (4/94)
645dff70c2f02fdd88c47b67/catalog.v1.parquet
645e0fb2c2f02fdd88c47b70/ (5/94)
645e0fb2c2f02fdd88c47b70/catalog.v1.parquet
645e3b2dc2f02fdd88c47b77/ (6/94)
645e3b2dc2f02fdd88c47b77/catalog.v1.parquet
645e3e4fc2f02fdd88c47b7a/ (7/94)
645e3e4fc2f02fdd88c47b7a/catalog.v1.parquet
64633fdfc2f02fdd88c47b85/ (8/94)
64633fdfc2f02fdd88c47b85/catalog.v1.parquet
6463589e59028dfdbee33336/ (9/94)
6463589e59028dfdbee33336/catalog.v1.parquet
6463716b59028dfdbee33338/ (10/94)
6463716b59028dfdbee33338/catalog.v1.parquet
646477ac59028dfdbee33341/ (11/94)
646477ac59028dfdbee33341/catalog.v1.parquet
6464899459028dfdbee33344/ (12/94)
6464899459028dfdbee33344/catalog.v1.parquet
64649f3f59028dfdbee3334a/ (13/94)
64649f3f59028dfdbee3334a/catalog.v1.parquet
646640b359028dfdbee3335c/ (14/94)
646640b359028dfdbee333

In [ ]:
minio_client.fget_object(os.getenv('BUCKET'), "67d182a43926af70561e6d36/catalog.v1.parquet.backup", f"kk/backup.parquet")

In [21]:
df = gpd.read_parquet(f"kk/backup.parquet")
df

,type,stac_version,stac_extensions,datetime,id,bbox,geometry,assets,links,repository
0,Feature,1.0.0,[],2025-03-25 13:03:20.212480,UCMerced.zip,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'de4455c5f375f3509f0f71...,[],eotdl


In [45]:
minio_client.fget_object(os.getenv('BUCKET'), "65f407248e68180349152973/catalog.v2.parquet", f"kk/backup.parquet")

In [46]:
df = gpd.read_parquet(f"kk/backup.parquet")
df

,type,stac_version,stac_extensions,datetime,id,bbox,geometry,assets,links,repository
0,Feature,1.0.0,[],2025-03-25 13:49:59.589565,model.onnx,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '99581a42cd56f66b1649f5...,[],eotdl


In [47]:
df.assets.sample(1).iloc[0]

{'asset': {'checksum': '99581a42cd56f66b1649f5246663cea7633a1007',
  'href': 'https://api.eotdl.com/models/65f407248e68180349152973/stage/model.onnx',
  'size': 130011868,
  'timestamp': '2024-05-30T17:14:39.987000'}}